# 01. Overview

#### 가설
* 가설1 : 거주 인구가 많은 지역에 스타벅스 매장이 많이 입지해 있을 것이다.
* 가설2 : 직장이 많은 지역에 스타벅스 매장이 많이 입지해 있을 것이다.
* 가설3 : 유동인구가 많은 지역에 스타벅스 매장이 많이 입지해 있을 것이다.
* 가설4 : 역 근처에 스타벅스 매장이 많이 입지해 있을 것이다.
* 가설5 : 땅값이 비싼지역에 스타벅스 매장이 많이 입지해 있을 것이다.
* 가설6 : 대학교 근처에 스타벅스 매장이 많이 입지해 있을것이다.
* 가설7 : 인근 카페가 많을 수록 스타벅스 매장이 많이 입지해 있을 것이다.
* 가설8 : 거주 인구가 20~30대가 많은 지역에 스타벅스 매장이 많이 입지해 있을 것이다.
* 가설9 : 교통편이 편리한(지하철, 버스 등) 지역에 스타벅스 매장이 많을 입지해 있을 것이다.

* 스타벅스 입지전략을 분석 : 가설1,2에 대한 검증 진행

#### 데이터 수집
* 1. 스타벅스 사이트에서 서울시에 존재하는 매장의 정보 수집
    * 매장명, 위도, 경도, 매장타입, 주소, 전화번호
    * https://www.starbucks.co.kr/

* 2. 서울시 주민등록 인구(구별)통계 정보 수집
    * 서울시 열린 데이터 광장
        * https://data.seoul.go.kr/dataList/419/S/2/datasetView.do
* 3. 서울시 사업체 현황(산업대분류별/동별) 통계 정보
    * 서울시 열린 데이터 광장
        * https://data.seoul.go.kr/dataList/104/S/2/datasetView.do

## 02.데이터 수집

### 2-1.서울시 스타벅스 매장 목록 데이터 수집:크롤링
* www.istarbucks.co.kr

In [12]:
import pyperclip
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(url)
driver.implicitly_wait(10)
seoul_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'

# WebDricerWait는 driver를 20초동안 대기시킴
# 조건 element_to_be_clickable : 클릭가능한 요소(seoul_btn)가 나타날때까지에 대한 조건
# 조건이 나타나면 seoul_btn를 클릭
try:
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR,seoul_btn))).click()
except TimeoutError:
    print('해당 페이지가 20초내에 열리지 않았음')

In [13]:
# '전체' 버튼 요소를 찾아 클릭
all_btn = '#mCSB_2_container > ul > li:nth-child(1) > a'
try:
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR,all_btn))).click()
except TimeoutError:
    print('해당 페이지가 20초내에 열리지 않았음')

In [15]:
# BeautifulSoup으로 HTML 파서 만들기
html = driver.page_source
soup = bs(html,'html.parser')

In [ ]:
#교수님버전
starbucks_soup_list = soup.select('li.quickResultLstCon')
print(len(starbucks_soup_list))
#강희 버전
st_li= soup.find('div',{'id':'mCSB_3_container'})
st_li.find_all('li',{'class':'quickResultLstCon'})#[0].text.strip().split('   ')

In [91]:
name = starbucks_soup_list[0].find('strong').text.strip() #건물명
lat = starbucks_soup_list[0]['data-lat'].strip() # 위도
lng = starbucks_soup_list[0]['data-long'].strip() # 경도
store_type = starbucks_soup_list[0].select('i')[0]['class'][0] #매장타입
add_tel = str(starbucks_soup_list[0].select('p.result_details')[0]).split('<br/>')
address = add_tel[0].split('>')[1]
tel = add_tel[1].split('<')[0]
print(name)
print(lat)
print(lng)
print(store_type)
print(address)
print(tel)

역삼아레나빌딩
37.501087
127.043069
pin_general
서울특별시 강남구 언주로 425 (역삼동)
1522-3232


In [84]:
starbucks_soup_list[0].select('i')[0]['class'][0] #매장타입

'pin_general'

In [80]:
print(starbucks_soup_list[0].select('i'))

[<i class="pin_general">리저브 매장 2번</i>]


In [ ]:
for i in 